In [1]:
import torchvision as tv
import phototour
import torch
from tqdm import tqdm
import numpy as np
import torch.nn as nn
import math

import tfeat_model
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# data import

In [2]:
lib_train = phototour.PhotoTour('.','liberty', download=True, train=True, mode = 'Quadruwithglobal', augment = True, nsamples=409600)
yos_train = phototour.PhotoTour('.','yosemite', download=True, train=True, mode = 'Quadruwithglobal', augment = True)
nd_train = phototour.PhotoTour('.','notredame', download=True, train=True, mode = 'Quadruwithglobal', augment = True)

# Found cached data ./liberty.pt
# Found cached data ./yosemite.pt
# Found cached data ./notredame.pt


# train

## 1. train : notredam & test : yosemite

In [3]:
eval_db = phototour.PhotoTour('.','yosemite', download=True, train=False)

train_db = nd_train
train_name = 'notredame'
eval_name = 'yosemite'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,batch_size=1024, shuffle=False,num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     print(loss)
    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+ eval_name+'_quadru_fpr.txt', np.array(fpr_per_epoch), delimiter=',')
    torch.save(tfeat.state_dict(), train_name + '-quadru.params')  

# Found cached data ./yosemite.pt


/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 30 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/home/cvlab/anaconda3/envs/pytorch/lib/python3.7/site-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 4, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
3334it [02:25, 22.87it/s]


0 0.149


3334it [02:25, 22.93it/s]


1 0.09446


...

97 0.05478


3334it [02:18, 24.15it/s]


98 0.05358


3334it [02:16, 24.46it/s]


99 0.05612


## 2. train : notredam & test : liberty

In [7]:
eval_db = phototour.PhotoTour('.','liberty', download=True, train=False)

train_db = nd_train
train_name = 'notredame'
eval_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,batch_size=1024, shuffle=False,num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

#     print(loss)
    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+ eval_name+'_quadru_fpr.txt', np.array(fpr_per_epoch), delimiter=',')
    torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')  

# Found cached data ./liberty.pt


3334it [02:17, 24.23it/s]


0 0.18854


3334it [02:19, 23.85it/s]


1 0.13324


...

97 0.05574


3334it [02:17, 24.19it/s]


98 0.0542


3334it [02:18, 24.13it/s]


99 0.05366


## 3. train : yosemite & test : notredam

In [5]:
eval_db = phototour.PhotoTour('.','notredame', download=True, train=False)

train_db = yos_train
eval_name = 'notredame'
train_name = 'yosemite'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+'-'+ eval_name+'_quadru_fpr.txt', np.array(fpr_per_epoch), delimiter=',')
    torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')                    

# Found cached data ./notredame.pt


3334it [02:18, 24.04it/s]


0 0.13216


3334it [02:16, 24.34it/s]


1 0.08032


...

97 0.05008


3334it [02:19, 23.92it/s]


98 0.04818


3334it [02:19, 23.85it/s]


99 0.05038


## 4. train : yosemite & test : liberty

In [8]:
eval_db = phototour.PhotoTour('.','liberty', download=True, train=False)

train_db = yos_train
eval_name = 'notredame'
train_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+'-'+ eval_name+'_quadru_fpr.txt', np.array(fpr_per_epoch), delimiter=',')
    torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')                    

# Found cached data ./liberty.pt


3334it [02:19, 23.94it/s]


0 0.23634


3334it [02:18, 24.06it/s]


1 0.18592


...

97 0.13188


3334it [02:18, 24.08it/s]


98 0.13022


3334it [02:18, 24.01it/s]


99 0.13542


## 5. train : liberty & test : yosemite

In [6]:
eval_db = phototour.PhotoTour('.','yosemite', download=True, train=False)

train_db = lib_train
eval_name = 'yosemite'
train_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+'-'+ eval_name+'_quadru_fpr.txt', np.array(fpr_per_epoch), delimiter=',')
    torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')                    

# Found cached data ./yosemite.pt


1366it [00:56, 24.24it/s]


0 0.20268


1366it [00:56, 24.24it/s]


1 0.15004


...

97 0.0687


1366it [00:57, 23.95it/s]


98 0.07016


1366it [00:57, 23.68it/s]


99 0.07216


## 6. train : liberty & test : notredame

In [9]:
eval_db = phototour.PhotoTour('.','notredame', download=True, train=False)

train_db = lib_train
eval_name = 'notredame'
train_name = 'liberty'

tfeat = tfeat_model.TNet()
tfeat = tfeat.cuda()

# this kind of works
optimizer = optim.Adam(tfeat.parameters(), lr=1e-3, betas=(0., 0.999))
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.8)

seed=42
torch.manual_seed(seed)
np.random.seed(seed)
# cv2.setRNGSeed(seed)

train_loader = torch.utils.data.DataLoader(train_db,
                                             batch_size=300, shuffle=False,
                                             num_workers=30)

eval_loader = torch.utils.data.DataLoader(eval_db,
                                             batch_size=1024, shuffle=False,

                                             num_workers=32)

fpr_per_epoch = []

for e in range(100):
    tfeat.train()
    for batch_idx, (data_a, data_p, data_n1, data_n2) in tqdm(enumerate(train_loader)):
        data_a = data_a.unsqueeze(1).float().cuda()
        data_p = data_p.unsqueeze(1).float().cuda()
        data_n1 = data_n1.unsqueeze(1).float().cuda()
        data_n2 = data_n2.unsqueeze(1).float().cuda()
        out_a, out_p, out_n1,out_n2 = tfeat(data_a), tfeat(data_p), tfeat(data_n1), tfeat(data_n2)

        hyperparameter = tfeat_model.newQuadruplet(margin1=2.0, margin2=1.0,gamma=0.8,ramda=0.8,t1=0.4,t2=0.2)
        loss=hyperparameter.loss(out_a,out_p,out_n1,out_n2)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    tfeat.eval()
    l = np.empty((0,))
    d = np.empty((0,))
    # evaluate the network after each epoch
    for batch_idx, (data_l, data_r, lbls) in enumerate(eval_loader):
        data_l = data_l.unsqueeze(1).float().cuda()
        data_r = data_r.unsqueeze(1).float().cuda()
        out_l, out_r = tfeat(data_l), tfeat(data_r)
        dists = torch.norm(out_l - out_r, 2, 1).detach().cpu().numpy()
        l = np.hstack((l, lbls.numpy()))
        d = np.hstack((d, dists))

    # FPR95 code from Yurun Tian
    d = torch.from_numpy(d)
    l = torch.from_numpy(l)
    dist_pos = d[l == 1]
    dist_neg = d[l != 1]
    dist_pos, indice = torch.sort(dist_pos)
    loc_thr = int(np.ceil(dist_pos.numel() * 0.95))
    thr = dist_pos[loc_thr]
    fpr95 = float(dist_neg.le(thr).sum()) / dist_neg.numel()
    print(e, fpr95)
    fpr_per_epoch.append([e, fpr95])
    scheduler.step()
    np.savetxt(train_name+'-'+ eval_name+'_quadru_fpr.txt', np.array(fpr_per_epoch), delimiter=',')
    torch.save(tfeat.state_dict(), train_name+'-'+eval_name + '-quadru.params')                    

# Found cached data ./notredame.pt


1366it [00:55, 24.47it/s]


0 0.1585


1366it [00:56, 24.06it/s]


1 0.12118


...

97 0.02132


1366it [00:56, 23.99it/s]


98 0.02126


1366it [00:57, 23.84it/s]


99 0.0221
